In [6]:
import pandas as pd

# read
df = pd.read_excel('relevance_test.xlsx')
# write
#df.to_excel('relevance_test.xlsx')
print(df)

ImportError: cannot import name 'is_numpy_dev' from 'pandas.compat' (C:\Users\Alberto\anaconda3\lib\site-packages\pandas\compat\__init__.py)

In [5]:
!pip install --upgrade numpy pandas

Requirement already up-to-date: numpy in c:\users\alberto\anaconda3\lib\site-packages (1.24.4)

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\Alberto\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5
